<a href="https://colab.research.google.com/github/poonam1261/Best-README-Template/blob/master/CDLabipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Shell Execution (you can rewrite the commands as per your need, eg. if you want to include a file as an input)
%%shell
yacc -d program.y
lex program.l
gcc y.tab.c lex.yy.c -ll
./a.out

program.l:16: bad character class
cc1: fatal error: lex.yy.c: No such file or directory
compilation terminated.
Enter a string: 

In [4]:
#@title Install *prerqeuisites* (run this cell first to work on LEX/YACC)
!sudo apt install flex bison

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfl-dev libfl2
Suggested packages:
  bison-doc flex-doc
The following NEW packages will be installed:
  bison flex libfl-dev libfl2
0 upgraded, 4 newly installed, 0 to remove and 30 not upgraded.
Need to get 1,072 kB of archives.
After this operation, 3,667 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 flex amd64 2.6.4-8build2 [307 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 bison amd64 2:3.8.2+dfsg-1build1 [748 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfl2 amd64 2.6.4-8build2 [10.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfl-dev amd64 2.6.4-8build2 [6,236 B]
Fetched 1,072 kB in 0s (7,171 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend can

In [23]:
!sudo apt-get update

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,099 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,542 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packag

In [51]:
#@title Writing YACC program
%%writefile program.y
%{
#include <stdio.h>
#include <stdlib.h>

void yyerror(const char *s);
int yylex();
%}

// Define token types
%token NUMBER

%%

program : expr '\n' { printf("Terminating\n");  exit(0); }
        ;
// Grammar rules for arithmetic expressions
expr    : expr '+' term    { printf("Valid Expression\n"); }
        | expr '-' term
        | term
        ;

term    : term '*' factor
        | term '/' factor
        | factor
        ;

factor  : factor '^' power
	| power
        ;
power   : NUMBER
 	| '(' expr ')' // Parentheses ensure correct precedence
 	;

%%

void yyerror(const char *s) {
    printf("Invalid Expression: %s\n", s);
}

int main() {
    printf("Enter an arithmetic expression: ");
    yyparse();
    return 0;
}




Overwriting program.y


In [52]:
#@title Writing Lex program
%%writefile program.l

%{
  #include "y.tab.h"

%}

digit [0-9]
operators [\+\-\*/\^]

%%

{digit} {yylval = atoi(yytext); return NUMBER;}
[+\-\*/\^] { return *yytext; }

[ \t\n] {}
. {return yytext[0];}



%%

int yywrap(){
  return 1;
}



Overwriting program.l


In [53]:
#@title Shell Execution (you can rewrite the commands as per your need, eg. if you want to include a file as an input)
%%shell
yacc -d program.y
lex program.l
gcc y.tab.c lex.yy.c -ll
./a.out

Enter an arithmetic expression: 3+2-4
Valid Expression
aaa
Invalid Expression: syntax error


In [25]:
%%shell
yacc -d program.y
lex program.l
gcc lex.yy.c y.tab.c -ll -o parser
./parser

Enter a string: ab
Invalid Character : b
String des not belong to the Grammar: syntax error


In [77]:
%%writefile program.l
%{
#include "y.tab.h"  // Include the Bison header for tokens
%}

%%
[0-9]+      { yylval = atoi(yytext); return NUM; }  // Match numbers and store the integer value
[+\-*/^]    { return yytext[0]; }  // Return operators as their respective tokens
[()]        { return yytext[0]; }  // Return parentheses as their respective tokens
[ \t\n]+    { /* Skip whitespace */ }
<<EOF>>     { return EOI; }  // End of input
%%

int yywrap() {
    return 1;  // Returns true when input ends
}



Overwriting program.l


In [78]:
%%shell
lex program.l

In [79]:
%%writefile parser.y

%{
  #include<stdio.h>
  #include<stdlib.h>
  #include<math.h>

  extern int yylex();
  extern int yyparse();
  extern FILE *yyin;

  void yyerror(const char *s){
    fprintf(stderr, "Syntax error: %s \n", s);
  }

%}



%token NUM
%token EOI
%left '+' '-'
%left '*' '/'
%left '^'
%left '(' ')'



%%
program : exp EOI {printf("The expression is valid, Answer = %d", $1); exit(0);}
exp : exp '+' exp {printf("%d + %d = %d", $1, $3, $1 + $3 ); $$ = $1 + $3;}
   | exp '-' exp {printf("%d - %d = %d", $1, $3, $1 - $3 ); $$ = $1 - $3;}
   | exp '*' exp {printf("%d * %d = %d", $1, $3, $1 * $3 ); $$ = $1 * $3;}
   | exp '/' exp {printf("%d / %d = %d", $1, $3, $1 / $3 ); $$ = $1 / $3;}
   | exp '^' exp {printf("%d ^ %d = %d", $1, $3, $1 ^ $3 ); $$ = $1 ^ $3;}
   | '(' exp ')' { $$ = $2; }
   | NUM { $$ = $1; }
   ;

%%

int main (){
  printf("Enter an expression: ");

  yyparse();

  return 0;
}

Overwriting parser.y


In [81]:
%%shell
yacc -d parser.y
gcc lex.yy.c y.tab.c -o parser
./parser

Enter an expression: 3+4 \n
\n4
Syntax error: syntax error 
3 + 4 = 7

In [33]:
%%writefile lexer.l
%option noyywrap
%{
#include "y.tab.h"  // includes token definitions (A, B)
%}

%%

a   { return A; }
\n  { return '\n'; }
.   { return -1; }  // invalid characters cause syntax error

%%



Overwriting lexer.l


In [40]:
%%writefile parser.y
%{
#include <stdio.h>
#include <stdlib.h>
int yylex(void);
void yyerror(const char *s);
%}

%token A B

%%

S : expr '\n' { printf("String is Valid\n");  exit(0); }
    ;
expr : A expr | ;

%%

int main() {
    printf("Enter a string of a's :\n");
    return yyparse();
}

void yyerror(const char *s) {
    fprintf(stderr, "Syntax Error: %s\n", s);
    exit(1);
}




Overwriting parser.y


In [ ]:
%%shell
lex lexer.l
bison -d parser.y
gcc lex.yy.c y.tab.c -o parser -lfl
./parser

Enter a string of a's followed by b's (a*b*):
aaa
Valid string: a*b*
